In [13]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import numpy as np

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6a1dd7bf12d24938b55e69a36c33dec7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZCw4mPNPhOmn2GpUI0FD59WfQuyCFwQMmusbgjVSpTeF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6a1dd7bf12d24938b55e69a36c33dec7.get_object(Bucket='agcwatsonstudioproject-donotdelete-pr-oahdwlxwjnxxjn',Key='UC2_Output.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,AG Number,Accused Name,Previous Antecedents?(CRO),TrialORPG?,Imprisonment (Years),Imprisonment (Months),Imprisonment (Weeks),Imprisonment (Days),Fine,Caning (Strokes),...,Drugtype1,Drugweight1,Genderofaccused,Generaltypeofoffence,Hsalabnumber1,Identitynumber,Nationality,Relateddrugtype,Reportnumber,Full offence
0,AG-123410-2017,DANIEL BIN FARHAN,Yes,Trial,7,0,0,0,0,0,...,methamphetamine,NaN,Male,Drug,AT-1733-10966-002-03,S9898605H,Singapore,NaN,E/20171214/2052,committed an offence under section 8(b)(ii) of...
1,AG-123436-2017,FARHAN BIN FARHAN,No,Trial,0,5,0,0,0,0,...,NaN,NaN,Male,Hurt,NaN,S9898939G,India,NaN,J/20170310/0023,committed anoffence punishable under section 3...
2,AG-900261-2019,MUHAMMAD FAZLI BIN AMAT SUNAR,Yes,Mention,0,10,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AG-123272-2019,MOHD FARHAN BIN ISMAIL,Yes,Mention,2,11,0,0,0,0,...,diamorphine,2.32 grams,Male,Drug,ID-1832-02893-001,S9898247H,Singapore,"['vegetable matter', 'vegetable matter']",F/20181210/2120,Section 8(a) of the Misuse of Drugs Act punish...
4,AG-123272-2019,MOHD FARHAN BIN ISMAIL,Yes,Mention,2,11,0,0,0,0,...,NaN,NaN,Male,Drug,NaN,S9898247H,Singapore,NaN,J/20181210/2116,committed an offence under regulation 15(3...


In [14]:
def preprocess_data(df):

    from nltk.stem import WordNetLemmatizer
    from collections import defaultdict
    from sklearn.compose import ColumnTransformer
    from gensim.parsing import preprocessing as pp
    from sklearn.preprocessing import LabelEncoder
    from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer,CountVectorizer
    import nltk
    
    nltk.download('stopwords')
    nltk.download('wordnet')



    df= df[df['TrialORPG?']=='Mention']
    df["Imprisonment"] = df['Imprisonment (Years)']*365 +df["Imprisonment (Months)"]*30 + df["Imprisonment (Weeks)"]*7+df["Imprisonment (Days)"]
    drop_cols = ['Imprisonment (Years)',"Imprisonment (Months)","Imprisonment (Weeks)","Imprisonment (Days)","DPP's Imprisonment (Years)","DPP's Imprisonment (Months)",
            "DPP's Imprisonment (Weeks)","DPP's Imprisonment (Days)","Reportnumber","Hsalabnumber1","Other Sentence","DPP's Other Sentence","DPP's Caning (Strokes)",
            "DPP's Fine"]
             
    df.drop(drop_cols,axis=1,inplace=True)
    new_column_names = ['AG Number','Accused Name','Previous Antecedents?(CRO)','TrialORPG?',"DPP's sentence (left to Court)",
                    'Firstpartofoffence','Offenceconnector','Secondpartofoffence','Full offence','Ageofaccused',
                    'Chargenumber','Dateofbirth','Dateofoffence','Drugtype1','Drugweight1','Genderofaccused','Generaltypeofoffence','Identitynumber','Nationality',
                    'Relateddrugtype','Imprisonment','Fine','Caning_Strokes']
    df = df.reindex(columns=new_column_names)

    def age_clean(cols):
        import re
        if type(cols) == str:
            match = re.search(r'\w+',cols)
            return int(match.group(0))

    df= df.reset_index(drop=True)

    df['Dateofbirth'] = df['Dateofbirth'].apply(lambda x:x[-4:] if type(x)==str else x)
    df['Dateofoffence'] = df['Dateofoffence'].apply(lambda x:x[-4:] if type(x)==str else x)

    df['Dateofbirth'] = df['Dateofbirth'].astype(float)
    df['Dateofoffence'] = df['Dateofoffence'].astype(float)

    df['Ageofaccused'] = df['Ageofaccused'].apply(age_clean)
    df['test'] = df['Ageofaccused']

    df['test'] =df['Dateofoffence']-df['Dateofbirth']
    df['Ageofaccused'].fillna(df.test, inplace=True)
    df.drop(['test','Dateofoffence','Dateofbirth','Firstpartofoffence','Offenceconnector','Secondpartofoffence','Accused Name'],axis=1,inplace=True)

    # ### Age manipulation

    age_dict= {"AG-123344-2018" : 28,"AG-123403-2019" : 52,"AG-123150-2018" : 26}
    charge_dict= {"DAC-918469-2019":52,"DAC-918470-2019":52,"DAC-901158-2019":52,"MAC-906285-2019":26,'MAC-906709-2018':28,"MAC-906708-2018":28}

    for key in charge_dict.keys():
        df.loc[df.Chargenumber == key,'Ageofaccused'] = charge_dict[key]
        
    df[df['Ageofaccused'] <20]

    dicts= {}
    from collections import Counter
    list_ages = set(df[df['Ageofaccused'].isna()]['AG Number'])
    unique_list_ages = list(list_ages)
    for columns in unique_list_ages:
        value_list= df[df['AG Number'] ==str(columns)]['Ageofaccused']
        b = Counter(value_list)
        value = b.most_common(1)[0][0]
        if not pd.isnull(value):
            dicts[columns] = value
    for key in dicts.keys():
        df.loc[df['AG Number'] == key,'Ageofaccused'] = dicts[key]

    #df['Full offence'][1]
    def clean_related_drug_type(cols):
        import re
        try:
            word1 = " ".join(re.findall("[a-zA-Z]+", str(cols)))
            return  " ".join(set(word1.split()))
        except Exception as e:
            print(e) 

    df['Relateddrugtype'] = df['Relateddrugtype'].replace(np.nan,"")
    df['Relateddrugtype'] = df['Relateddrugtype'].apply(clean_related_drug_type)

    df['Drugweight1']= df['Drugweight1'].apply(age_clean)
    numerical_feature_mask = df.dtypes!='object'
    numerical_cols = df.columns[numerical_feature_mask].tolist()
    df.loc[df.Ageofaccused.isna(), 'Ageofaccused'] = df['Ageofaccused'].median()
    df[numerical_cols] = df[numerical_cols].fillna(0)
    df['Full offence'] = df['Full offence'].replace(np.nan," ")
    
    return df


def feature_creation(clean_df):
    # ### Text Cleaning and Transformation

    #clean_df = feature_df.copy()
    from nltk.stem import WordNetLemmatizer
    from collections import defaultdict
    from sklearn.compose import ColumnTransformer
    from gensim.parsing import preprocessing as pp
    from sklearn.preprocessing import LabelEncoder
    from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer,CountVectorizer
    import nltk
    
    nltk.download('stopwords')
    nltk.download('wordnet')

    lemmatizer = WordNetLemmatizer()
    
    def age_clean(cols):
        import re
        if type(cols) == str:
            match = re.search(r'\w+',cols)
            return int(match.group(0))



    pp_list_new = [
        lambda x: x.lower(),
        pp.strip_tags,
        pp.strip_multiple_whitespaces,
        pp.strip_punctuation,
        pp.strip_short,
        pp.remove_stopwords,
        pp.strip_numeric
              ]

    def test_tokenizer(line):
        tokens = pp.preprocess_string(line, filters=pp_list_new)
        tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens]
        return "  ".join(tokens)


    def labelencode(clean_df):
        for f in clean_df.columns:
            if clean_df[f].dtype == 'object' and f not in ['Full offence',"Relateddrugtype"]:
                label = LabelEncoder()
                label.fit(list(clean_df[f].values))
                clean_df[f] = label.transform(list(clean_df[f].values))
        return clean_df



    clean_df= clean_df.drop(['AG Number','Chargenumber','Identitynumber','TrialORPG?'],axis=1)
    clean_df['Caning_Strokes'] = clean_df['Caning_Strokes'].replace(np.nan,0)
    clean_df['Drugweight1']= clean_df['Drugweight1'].apply(age_clean)

    clean_df['Full offence'] = clean_df['Full offence'].apply(lambda x:test_tokenizer(x))

    clean_df = labelencode(clean_df)

    vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2',ngram_range=(1,1))
    #vectorizer = HashingVectorizer(n_features=10000,norm=None,alternate_sign=False) 

    offence_vec = vectorizer.fit_transform(clean_df['Full offence'])

    df1 = pd.DataFrame(offence_vec.toarray(), columns=vectorizer.get_feature_names())

    clean_df.drop(['Full offence'], axis=1, inplace=True)
    clean_df = pd.concat([clean_df, df1], axis=1)

    ## Related Drug Feature Vectorizer
    offence_vec_rdt = vectorizer.fit_transform(clean_df['Relateddrugtype'])
    print(vectorizer.inverse_transform(offence_vec_rdt[1]))
    df2 = pd.DataFrame(offence_vec_rdt.toarray(), columns=vectorizer.get_feature_names())
    clean_df = pd.concat([clean_df, df2], axis=1)
    clean_df.drop(['Relateddrugtype'], axis=1, inplace=True)
    return clean_df


    
def model_train_test(model_df):

    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline
    from xgboost import XGBRegressor
    from sklearn.multioutput import MultiOutputRegressor
    from sklearn.linear_model import Ridge
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedKFold,KFold,RepeatedStratifiedKFold
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import AdaBoostRegressor
    from numpy import absolute
    from numpy import mean
    from numpy import std
    import lightgbm as lgb
    from lightgbm import LGBMRegressor

    # ## Model Training ,Cross Validation & Testing
    target = ['Imprisonment','Fine','Caning_Strokes']
    X = model_df.drop(target, axis=1) 
    Y = model_df[target]

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size= 0.20, random_state=42)
 
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

    model = XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
    
    ridge = Ridge()
    rfregressor = RandomForestRegressor()
    knnregressor = KNeighborsRegressor()
    extratrees = ExtraTreesRegressor()
    Gbm = GradientBoostingRegressor()
    adaboost = AdaBoostRegressor()
    lgbregressor = LGBMRegressor(random_state=101)
    models = [model,ridge,rfregressor,lgbregressor,knnregressor]

    score_dicts = {}
    feature_imps = {}

    for each in models:
        try:
            mor_model = MultiOutputRegressor(each)
            n_scores = cross_val_score(mor_model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

            mor_model.fit(X_train,Y_train)
            predictions = mor_model.predict(X_test)
            score = r2_score(Y_test,predictions)

            #feature_importance

            feature_imp = pd.DataFrame(sorted(zip(mor_model.estimators_[0].feature_importances_,X.columns)), columns=['Value','Feature'])

            # force the scores to be positive

            n_scores = absolute(n_scores)

            # Score_dict
            each = str(type(each))
            each = each.split(".")[-1]
            score_dicts[each] = score
            feature_imps[each] = feature_imp
            
        except Exception as e:
            pass

    return score_dicts,feature_imps








In [15]:
clean_df_new = preprocess_data(df_data_1)
model_df = feature_creation(clean_df_new)
score_dicts,feature_imps = model_train_test(model_df)
score_dicts

[array(['matter', 'vegetable'], dtype='<U11')]


{"XGBRegressor'>": 0.440579468090242,
 "RandomForestRegressor'>": 0.7278411432590616,
 "LGBMRegressor'>": 0.8239600338063492}